In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    std_W1 = torch.sqrt(torch.tensor(1. / d0))
    W1 =0.2* torch.empty(d1, d0, device=device).normal_(0, std_W1)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    std_W2 = torch.sqrt(torch.tensor(1. / d1))
    W2 =0.2* torch.empty(d2, d1, device=device).normal_(0, std_W2)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    std_W3 = torch.sqrt(torch.tensor(1. / d2))
    W3 =0.2* torch.empty(d3, d2, device=device).normal_(0, std_W3)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    std_W4 = torch.sqrt(torch.tensor(1. / d3))
    W4 =0.2* torch.empty(d4, d3, device=device).normal_(0, std_W4)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    std_W5 = torch.sqrt(torch.tensor(1. / d4))
    W5 =0.2* torch.empty(d5, d4, device=device).normal_(0, std_W5)
    b5 = 0*torch.ones(d5, 1, device=device)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    std_W6 = torch.sqrt(torch.tensor(1. / d5))
    W6 =0.2* torch.empty(d6, d5, device=device).normal_(0, std_W6)
    b5 = 0*torch.ones(d5, 1, device=device)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "LecunNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 4.330068826675415 - sq_loss: 661.6827392578125 - tot_loss: 967.5189884508873 - acc: 0.16825353645266594 - val_acc: 0.16762809636918902
Repeatition 1 Epoch 2 / 500 
 - time: 3.783352851867676 - sq_loss: 294.0812072753906 - tot_loss: 493.16393100330606 - acc: 0.1761425462459195 - val_acc: 0.168306752629793
Repeatition 1 Epoch 3 / 500 
 - time: 3.7504944801330566 - sq_loss: 164.3061981201172 - tot_loss: 269.82554174400866 - acc: 0.18049510337323177 - val_acc: 0.167288768238887
Repeatition 1 Epoch 4 / 500 
 - time: 3.6591269969940186 - sq_loss: 89.59539794921875 - tot_loss: 152.4732067878358 - acc: 0.20239390642002175 - val_acc: 0.19273837801153715
Repeatition 1 Epoch 5 / 500 
 - time: 3.742438793182373 - sq_loss: 48.28154754638672 - tot_loss: 89.32036096276715 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 3.76977

Repeatition 1 Epoch 48 / 500 
 - time: 3.255863904953003 - sq_loss: 0.00043795761303044856 - tot_loss: 1.6077959999092855 - acc: 0.573993471164309 - val_acc: 0.5466576179165252
Repeatition 1 Epoch 49 / 500 
 - time: 3.2832179069519043 - sq_loss: 0.000410299253417179 - tot_loss: 1.5474748185788485 - acc: 0.5885473340587595 - val_acc: 0.5541228367831693
Repeatition 1 Epoch 50 / 500 
 - time: 3.325080394744873 - sq_loss: 0.0003849121567327529 - tot_loss: 1.4965629720227298 - acc: 0.6039173014145811 - val_acc: 0.5622667119104173
Repeatition 1 Epoch 51 / 500 
 - time: 3.3134214878082275 - sq_loss: 0.00036144498153589666 - tot_loss: 1.4498755530148628 - acc: 0.6232317736670294 - val_acc: 0.5731252120800815
Repeatition 1 Epoch 52 / 500 
 - time: 3.315077543258667 - sq_loss: 0.0003397279360797256 - tot_loss: 1.4030300931481179 - acc: 0.6433623503808488 - val_acc: 0.5877163216830675
Repeatition 1 Epoch 53 / 500 
 - time: 3.193661689758301 - sq_loss: 0.0003195462340954691 - tot_loss: 1.372550892

Repeatition 1 Epoch 95 / 500 
 - time: 4.134057998657227 - sq_loss: 3.267559804953635e-05 - tot_loss: 0.44424647776349957 - acc: 0.8759521218715995 - val_acc: 0.8646080760095012
Repeatition 1 Epoch 96 / 500 
 - time: 4.150705575942993 - sq_loss: 3.106782241957262e-05 - tot_loss: 0.4422376677136981 - acc: 0.8775843307943417 - val_acc: 0.8656260604004072
Repeatition 1 Epoch 97 / 500 
 - time: 5.869747161865234 - sq_loss: 2.9569220714620315e-05 - tot_loss: 0.4288485581882924 - acc: 0.8782644178454843 - val_acc: 0.8666440447913132
Repeatition 1 Epoch 98 / 500 
 - time: 5.681869268417358 - sq_loss: 2.8119064154452644e-05 - tot_loss: 0.41895821681146117 - acc: 0.8792165397170838 - val_acc: 0.8680013573125213
Repeatition 1 Epoch 99 / 500 
 - time: 5.4727253913879395 - sq_loss: 2.6725696443463676e-05 - tot_loss: 0.40978872604591743 - acc: 0.8807127312295974 - val_acc: 0.8696979979640312
Repeatition 1 Epoch 100 / 500 
 - time: 4.3922929763793945 - sq_loss: 2.5384129912708886e-05 - tot_loss: 0.4

Repeatition 1 Epoch 141 / 500 
 - time: 6.067206382751465 - sq_loss: 4.183798409940209e-06 - tot_loss: 0.24197582162520348 - acc: 0.9140369967355821 - val_acc: 0.9080420766881574
Repeatition 1 Epoch 142 / 500 
 - time: 6.1403138637542725 - sq_loss: 4.0437516872771084e-06 - tot_loss: 0.24114397710329172 - acc: 0.9151251360174102 - val_acc: 0.9087207329487614
Repeatition 1 Epoch 143 / 500 
 - time: 6.197977066040039 - sq_loss: 3.9174315134005155e-06 - tot_loss: 0.2479293088562855 - acc: 0.9160772578890098 - val_acc: 0.9090600610790635
Repeatition 1 Epoch 144 / 500 
 - time: 6.11615777015686 - sq_loss: 3.7939471440040506e-06 - tot_loss: 0.23648041317107982 - acc: 0.9166213275299239 - val_acc: 0.9107567017305734
Repeatition 1 Epoch 145 / 500 
 - time: 6.067431211471558 - sq_loss: 3.6687765714304987e-06 - tot_loss: 0.24599070481416163 - acc: 0.9170293797606094 - val_acc: 0.9107567017305734
Repeatition 1 Epoch 146 / 500 
 - time: 6.072753190994263 - sq_loss: 3.555972170943278e-06 - tot_loss:

Repeatition 1 Epoch 187 / 500 
 - time: 4.919192314147949 - sq_loss: 1.587561087035283e-06 - tot_loss: 0.1765225948933562 - acc: 0.9378400435255713 - val_acc: 0.9307770614183916
Repeatition 1 Epoch 188 / 500 
 - time: 4.982091188430786 - sq_loss: 1.5733000964246457e-06 - tot_loss: 0.1783394311001718 - acc: 0.9383841131664853 - val_acc: 0.9311163895486936
Repeatition 1 Epoch 189 / 500 
 - time: 4.912284851074219 - sq_loss: 1.559342308610212e-06 - tot_loss: 0.19886950794602676 - acc: 0.9389281828073993 - val_acc: 0.9311163895486936
Repeatition 1 Epoch 190 / 500 
 - time: 4.921783208847046 - sq_loss: 1.5470701555386768e-06 - tot_loss: 0.1986580704706693 - acc: 0.9393362350380848 - val_acc: 0.9317950458092976
Repeatition 1 Epoch 191 / 500 
 - time: 4.974924087524414 - sq_loss: 1.5285953622878878e-06 - tot_loss: 0.1820906056034648 - acc: 0.9396082698585418 - val_acc: 0.9321343739395996
Repeatition 1 Epoch 192 / 500 
 - time: 4.9469099044799805 - sq_loss: 1.5116245322133182e-06 - tot_loss: 0

...

Repeatition 1 Epoch 279 / 500 
 - time: 5.111647129058838 - sq_loss: 1.020293552755902e-06 - tot_loss: 0.17350502224195274 - acc: 0.9650435255712732 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 280 / 500 
 - time: 5.02737283706665 - sq_loss: 1.0188218766415957e-06 - tot_loss: 0.16230862049734895 - acc: 0.9657236126224157 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 281 / 500 
 - time: 5.110271453857422 - sq_loss: 1.0158923942071851e-06 - tot_loss: 0.16477605197448764 - acc: 0.9655875952121872 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 282 / 500 
 - time: 5.021154165267944 - sq_loss: 1.0140723816220998e-06 - tot_loss: 0.17295974010695447 - acc: 0.9659956474428727 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 283 / 500 
 - time: 5.177645206451416 - sq_loss: 1.0125756944034947e-06 - tot_loss: 0.17369816573429508 - acc: 0.9659956474428727 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 284 / 500 
 - time: 5.02069354057312 - sq_loss: 1.0108361720995163e-06 - tot_loss:

Repeatition 1 Epoch 325 / 500 
 - time: 5.263864994049072 - sq_loss: 9.166729455500899e-07 - tot_loss: 0.167252171435452 - acc: 0.970892274211099 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 326 / 500 
 - time: 5.247008562088013 - sq_loss: 9.147104265139205e-07 - tot_loss: 0.16791938951045893 - acc: 0.970892274211099 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 327 / 500 
 - time: 5.026521444320679 - sq_loss: 9.137548317994515e-07 - tot_loss: 0.1664187764132916 - acc: 0.9710282916213275 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 328 / 500 
 - time: 5.131076812744141 - sq_loss: 9.114557428802073e-07 - tot_loss: 0.16682159610895697 - acc: 0.971164309031556 - val_acc: 0.9558873430607397
Repeatition 1 Epoch 329 / 500 
 - time: 5.078932523727417 - sq_loss: 9.098445730160165e-07 - tot_loss: 0.15990818809534657 - acc: 0.971164309031556 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 330 / 500 
 - time: 5.440105438232422 - sq_loss: 9.0811244035649e-07 - tot_loss: 0.179616855

Repeatition 1 Epoch 371 / 500 
 - time: 5.113956451416016 - sq_loss: 8.37725451674487e-07 - tot_loss: 0.16857618684724307 - acc: 0.9749727965179543 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 372 / 500 
 - time: 5.288428544998169 - sq_loss: 8.362300718545157e-07 - tot_loss: 0.16341633833931635 - acc: 0.9747007616974973 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 373 / 500 
 - time: 5.105812311172485 - sq_loss: 8.341810371348402e-07 - tot_loss: 0.1642130994766724 - acc: 0.9749727965179543 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 374 / 500 
 - time: 5.209012746810913 - sq_loss: 8.330495120389969e-07 - tot_loss: 0.17089215823184922 - acc: 0.9749727965179543 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 375 / 500 
 - time: 5.090527296066284 - sq_loss: 8.317161928061978e-07 - tot_loss: 0.18074776977764961 - acc: 0.9751088139281828 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 376 / 500 
 - time: 5.128396272659302 - sq_loss: 8.307885650538083e-07 - tot_loss: 0.14

Repeatition 1 Epoch 417 / 500 
 - time: 3.291700601577759 - sq_loss: 7.717801508988487e-07 - tot_loss: 0.17616846225405491 - acc: 0.9774211099020674 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 418 / 500 
 - time: 3.157927989959717 - sq_loss: 7.703470714659488e-07 - tot_loss: 0.16468562000442089 - acc: 0.9775571273122959 - val_acc: 0.9572446555819477
Repeatition 1 Epoch 419 / 500 
 - time: 3.11499285697937 - sq_loss: 7.685841865168186e-07 - tot_loss: 0.15309146422807407 - acc: 0.9774211099020674 - val_acc: 0.9572446555819477
Repeatition 1 Epoch 420 / 500 
 - time: 3.2452609539031982 - sq_loss: 7.66191703860386e-07 - tot_loss: 0.1459701050178177 - acc: 0.9774211099020674 - val_acc: 0.9572446555819477
Repeatition 1 Epoch 421 / 500 
 - time: 3.263324022293091 - sq_loss: 7.653923148609465e-07 - tot_loss: 0.1794693317551932 - acc: 0.977829162132753 - val_acc: 0.9565659993213438
Repeatition 1 Epoch 422 / 500 
 - time: 3.20737624168396 - sq_loss: 7.637536327820271e-07 - tot_loss: 0.17195

Repeatition 1 Epoch 463 / 500 
 - time: 3.237839698791504 - sq_loss: 7.203234986263851e-07 - tot_loss: 0.17510379877539806 - acc: 0.9795973884657236 - val_acc: 0.9575839837122497
Repeatition 1 Epoch 464 / 500 
 - time: 3.2346651554107666 - sq_loss: 7.194009867816931e-07 - tot_loss: 0.16665092495099754 - acc: 0.9797334058759521 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 465 / 500 
 - time: 3.1749985218048096 - sq_loss: 7.181292858149391e-07 - tot_loss: 0.17754608253861992 - acc: 0.9795973884657236 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 466 / 500 
 - time: 3.1216328144073486 - sq_loss: 7.173279072958394e-07 - tot_loss: 0.18008419872700188 - acc: 0.9797334058759521 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 467 / 500 
 - time: 3.1592862606048584 - sq_loss: 7.15594069333747e-07 - tot_loss: 0.17246394114743246 - acc: 0.9797334058759521 - val_acc: 0.9569053274516457
Repeatition 1 Epoch 468 / 500 
 - time: 3.1741292476654053 - sq_loss: 7.148009331103822e-07 - tot_loss

In [8]:
seed

10